# Predecir ingresos por persona

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def num_or_cat(df):
    categorical = df.select_dtypes(include='object').columns
    numerical = df.select_dtypes(exclude='object').columns

    return(categorical, numerical)

In [ ]:
def remove_outliers(df, numeric_columns):
    df_filtered = df.copy()
    for column_name in numeric_columns:
        if pd.api.types.is_numeric_dtype(df[column_name]):
            q1 = df[column_name].quantile(0.25)
            q3 = df[column_name].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            df_filtered = df_filtered[(df_filtered[column_name] >= lower_bound) & (df_filtered[column_name] <= upper_bound)]
    return df_filtered

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
dataframe = pd.read_csv('./encuesta-anual-hogares-2018.csv', encoding='latin1')
dataframe.head(10)

In [ ]:
#categorical_cols, numerical_cols= num_or_cat(dataframe)
#categorical_cols

In [ ]:
#dataframe = remove_outliers(dataframe, numerical_cols)

In [ ]:
dataframe.info()

se hace un info para ver los valores faltantes, se procede a ir reemplazando

# Reemplazo de valores faltantes

un array con las columnas con valores faltantes para ir teniendo de referencia
en todas las columnas salvo nivel_max_educativo se descartaron los datos debido a que la cantidad faltante era bastante baja en comparacion con el tamaño del dataset, otro problema que se presenta para reemplazar es que no se encontro un valor especifico por el cual reemplazar estos valores

### nivel_max_educativo


In [ ]:
def columnas_nan(df):
    columnas_val_faltantes = df.columns[df.isnull().any()].tolist()
    
    if columnas_val_faltantes:
        print(f"Columnas con nan: {', '.join(columnas_val_faltantes)}")
    else:
        print("clean")

In [ ]:
columnas_nan(dataframe)

# hijos_nac_vivos

esta columna tenia aprox 5 mil valores faltantes pero al compararla con la columna de al lado cantidad_hijos_nac_vivos se veia que coincidian los valores faltantes con las personas que tenian 0 (no corresponde antes del cambio). por lo cual se llenaron los valores faltantes con 'No'

+ ((dataframe['hijos_nacidos_vivos'].isnull()) & (dataframe['cantidad_hijos_nac_vivos'] == 0)).value_counts() este filtro fue util para revisar si el cambio era correcto

In [ ]:
dataframe[dataframe['hijos_nacidos_vivos'].isna()]
dataframe['hijos_nacidos_vivos'] = dataframe['hijos_nacidos_vivos'].fillna('No')

# cantidad_hijos_nacidos_vivos

para la cantidad de hijos nacidos se asume que 0 equivale a no corresponde ya que coincidia con las filas de la columna hijos_nacidos_vivos cuando esta era igual a No. Por lo que se va a cambiar por 0, y se va a castear la columna a int ya que cambiando este valor toda la columna tiene valores numericos.
Luego de esto solo quedan dos valores faltantes los cuales se descartan

In [ ]:
dataframe[dataframe['cantidad_hijos_nac_vivos'].isna()]

In [ ]:
dataframe.dropna(subset =['cantidad_hijos_nac_vivos'], inplace=True)

In [ ]:
reemplazo = {"No corresponde": 0}
dataframe['cantidad_hijos_nac_vivos'] = dataframe['cantidad_hijos_nac_vivos'].replace(reemplazo)
dataframe['cantidad_hijos_nac_vivos'] = dataframe['cantidad_hijos_nac_vivos'].astype(int)


# Situacion conyugal

Se descarto el valor en esta columna

In [ ]:
dataframe['situacion_conyugal'].unique()

#dataframe[dataframe['situacion_conyugal'].isnull()] ['edad'].value_counts()

In [ ]:
dataframe['situacion_conyugal'].value_counts()
dataframe['situacion_conyugal'].isna().sum()

In [ ]:
dataframe.dropna(subset =['situacion_conyugal'], inplace=True)

# Sector educativo

a esta variable le faltan solo dos datos los cuales seran descartados

In [ ]:
dataframe['sector_educativo'].unique()

In [ ]:
dataframe['sector_educativo'].value_counts()

dataframe[dataframe['sector_educativo'].isnull()] ['edad'].value_counts()

In [ ]:
dataframe['sector_educativo'] = dataframe['sector_educativo'].fillna('No corresponde')

# nivel_max_educativo
este es el que tendria que ver porque le faltan 1000 valores aprox

In [ ]:
dataframe['nivel_max_educativo'].unique()

In [ ]:
dataframe['nivel_max_educativo'].value_counts()

In [ ]:
dataframe['nivel_max_educativo'].isna().sum()

In [ ]:
dataframe.loc[(dataframe['nivel_max_educativo'].isnull()) & (dataframe['edad'] >= 0) & (dataframe['edad'] <= 5), "nivel_max_educativo"] = "No corresponde"

In [ ]:
dataframe[dataframe['nivel_max_educativo'].isnull()] ['edad'].value_counts()
dataftame = dataframe[dataframe['nivel_max_educativo'].isnull() ]

In [ ]:
dataframe.dropna(subset =['nivel_max_educativo'], inplace=True)

# años_escolaridad
a esta variable se le cambia ningun año aprobado por 0 para despues poder castear la columna a enteros

tengo 85 valores faltantes, al no ser una cantidad tan significativa seran descartados

In [ ]:
dataframe['años_escolaridad'].isna().sum()

In [ ]:
dataframe['años_escolaridad'] = dataframe['años_escolaridad'].replace("Ningun año de escolaridad aprobado", 0)

In [ ]:
dataframe['años_escolaridad'].unique()

In [ ]:
dataframe.dropna(subset =['años_escolaridad'], inplace=True)
dataframe['años_escolaridad'] = dataframe['años_escolaridad'].astype(int)

In [ ]:
dataframe['años_escolaridad'].isna().sum()

# lugar_nacimiento

en esta variable tambien solo falta un valor el que se procede a descartarse

In [ ]:
dataframe['lugar_nacimiento'].unique()

In [ ]:
dataframe['lugar_nacimiento'].value_counts()

In [ ]:
dataframe['lugar_nacimiento'].isna().sum()

In [ ]:
dataframe.dropna(subset =['lugar_nacimiento'], inplace=True)

# afiliacion_salud
al igual que la variable anterior solo falta un valor y se va a descartar


In [ ]:
dataframe['afiliacion_salud'].unique()

In [ ]:
dataframe['afiliacion_salud'].value_counts()

In [ ]:
dataframe['afiliacion_salud'].isna().sum()

In [ ]:
dataframe.dropna(subset=['afiliacion_salud'], inplace=True)

# Analisis de datos

In [ ]:
categorical_cols, numerical_cols= num_or_cat(dataframe)
categorical_cols

In [ ]:
dataframe = remove_outliers(dataframe, numerical_cols)

In [ ]:
dataframe.head()

In [ ]:
dataframe.describe()

In [ ]:
dataframe.info()

In [ ]:
sns.countplot(x=dataframe['nivel_max_educativo'])
plt.xticks(rotation=45)
plt.show()

## Boxplot ingresos en funcion de años de escolaridad
con este grafico podemos observar que a medida que aumentan los años de escolaridad *parecen* aumentan los ingresos total laborables

In [ ]:
sns.boxplot(x=dataframe['años_escolaridad'], y=dataframe['ingreso_total_lab'])

## Boxplot ingreso total laborable
+ La distribucion de los datos es asimetrica hacia la derecha
esto tiene sentido ya que hay varias personas en el dataset que no trabajan o no tienen edad para trabajar (ponele), 
+ podemos ver que la media es de aproximadamente 13300 pesos, lo cual se aproxima al salario minimo de 2018 (11 mil pesos)

http://www.siasueldos.com/salario-m%C3%ADnimo-vital-y-m%C3%B3vil-actualizaci%C3%B3n-de-montos-septiembre-y-diciembre-2018#:~:text=24%20Septiembre%202018-,SALARIO%20M%C3%8DNIMO%20VITAL%20Y%20M%C3%93VIL%2DACTUALIZACI%C3%93N%20DE%20MONTOS%2DSEPTIEMBRE%20Y,%3A%20%24%2011.300%2C%2D%20mensuales.

+ Encontramos muchos valores atipicos lo que representa que los sueldos mas altos le corresponden a pocsas personas y la mayoria de personas tiene sueldos mas bajos

In [ ]:
dataframe['ingreso_total_lab'].describe()

In [ ]:
#dataframe['ingreso_total_lab_log'] = np.log(dataframe['ingreso_total_lab'])
sns.boxplot(x=dataframe['ingreso_total_lab'])
plt.title('Box Plot de ingreso total')
plt.xlabel('Value')
plt.show()

## Vemos la escolaridad podemos ver que en promedio las personas tienen 12 años de escolaridad lo que equivaldria a un secundario completo
+ con un describe vemos que la media esta mas cerca de 0 que la mediana, esto se debe a que hay personas que aun son muy jovenes como para ir a la escuela como tambien hay gente que no asistio o no la termino

In [ ]:
dataframe['años_escolaridad'].describe()

In [ ]:
plt.hist(dataframe['años_escolaridad'], bins=10, edgecolor='black')
plt.title('Histograma de escolaridad')
plt.xlabel('Valor')
plt.ylabel('frecuencia')
plt.show()

In [ ]:
plt.hist(dataframe['edad'], bins=20, edgecolor='black')
plt.title('Histograma de edades')
plt.xlabel('Valor')
plt.ylabel('frecuencia')
plt.show()

## Hijos_nacidos_vivos
+ El nombre de esta variable es un poco confuso, se puede asumir que se refiere a partos
+ La idea es que No representa que la persona no tiene hijos
+ Si representa que si tiene hijos

In [ ]:
sns.countplot(x='hijos_nacidos_vivos', data=dataframe)

## Dominio
Usamos este plot para ver como se distribuye la gente si en partes de la ciudad o una villa de emergencia

In [ ]:
sns.countplot(x='dominio', data=dataframe)

In [ ]:
cantidad_situacion_conyugal = dataframe['situacion_conyugal'].value_counts()

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(cantidad_situacion_conyugal, labels=cantidad_situacion_conyugal.index, autopct='%1.1f%%')
plt.title('Pie Chart de situacion conyugal')
plt.show()

In [ ]:
sns.boxplot(x=dataframe['edad'], y=dataframe['situacion_conyugal'])

In [ ]:
cantidad_afiliacion_salud = dataframe['afiliacion_salud'].value_counts()

## afiliacion_salud
+ vemos la proporcion de personas que tienen cada servicio
+ despues hacemos un boxplot en funcion de los ingresos para ver si hay alguna relacion entre estas variables

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(cantidad_afiliacion_salud, labels=cantidad_afiliacion_salud.index, autopct='%1.1f%%')
plt.title('Pie Chart de afiliacion salud')
plt.show()

In [ ]:
sns.boxplot(x=dataframe['ingreso_total_lab'], y=dataframe['afiliacion_salud'])
plt.title("Grafico de Afiliacion_salud en funcion de los ingresos")

In [ ]:
dataframe.head()

# correlaciones
en las variables numericas

In [ ]:
sns.heatmap(dataframe[['ingreso_total_lab', 'edad','años_escolaridad', 'ingreso_total_no_lab']].corr(), annot=True)

# Crear dummies

## Codear datos categoricos

+ ver que variables son numericas o categoricas

In [ ]:
def num_or_cat(df):
    categorical = df.select_dtypes(include='object').columns
    numerical = df.select_dtypes(exclude='object').columns

    return(categorical, numerical)

In [ ]:
categorical_cols, numerical_cols= num_or_cat(dataframe)
categorical_cols

### Codear las booleanas 
una lista (las que tienen valores 'Si' 'No', hombre mujer), despues de que las tengo las busco

In [ ]:
booleanas = []
for col in categorical_cols:
    if len(dataframe[col].unique()) == 2:
        booleanas.append(col)
booleanas

In [ ]:
dominio_dummie = pd.get_dummies(dataframe['dominio']).astype(int)
hijos_nacidos_vivos_dummie = pd.get_dummies(dataframe['hijos_nacidos_vivos']).astype(int)
sexo_dummie = pd.get_dummies(dataframe['sexo']).astype(int)

In [ ]:
dataframe = pd.concat([dataframe, dominio_dummie], axis=1)
dataframe = pd.concat([dataframe, hijos_nacidos_vivos_dummie], axis=1)
dataframe = pd.concat([dataframe, sexo_dummie], axis=1)

In [ ]:
dataframe = dataframe.drop(labels='dominio',axis=1)
dataframe = dataframe.drop(labels='hijos_nacidos_vivos',axis=1)
dataframe = dataframe.drop(labels='sexo',axis=1)

In [ ]:
dataframe.info()

### Codear situacion conyugal

In [ ]:
situacion_conyugal_dummie = pd.get_dummies(dataframe['situacion_conyugal']).astype(int)

In [ ]:
dataframe = pd.concat([dataframe, situacion_conyugal_dummie], axis=1)

In [ ]:
dataframe = dataframe.drop(labels='situacion_conyugal',axis=1)

### afiliacion salud

In [ ]:
afiliacion_salud_dummie = pd.get_dummies(dataframe['afiliacion_salud']).astype(int)

In [ ]:
dataframe = pd.concat([dataframe, afiliacion_salud_dummie], axis=1)

In [ ]:
dataframe = dataframe.drop(labels='afiliacion_salud',axis=1)

In [ ]:
dataframe.columns

In [ ]:
dataframe = dataframe.rename(columns={'Villas de emergencia': 'villas_de_emergencia',
                                       'Resto de la Ciudad': 'resto_de_la_ciudad',
                                       "No":"hijos_nacidos_no",
                                       "Si":"hijos_nacidos_si",
                                       "Mujer":"sexo_mujer",
                                       "Varon":"sexo_varon",
                                       "Casado/a":"casado_a",
                                       "Divorciado/a":"divorciado_a",
                                       'No corresponde': 'no_corresponde',
                                       "Separado/a de unión o matrimonio":"separado_a_de_union_o_matrimonio",
                                       "Soltero/a":"soltero_a",
                                       "Unido/a":"unido_a",
                                       "Viudo/a":"viudo_a",
                                       "Solo obra social":"Solo_obra_social",
                                       "Solo plan de medicina prepaga por contratación voluntaria":"solo_plan_de_medicina_prepaga_por_contratacion_voluntaria",
                                       "Solo prepaga o mutual via OS":"solo_prepaga_o_mutual_via_OS",
                                       "Solo sistema publico":"solo_sistema_publico",
                                       })


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
numerical_cols

# normalizacion de datos numericos

In [ ]:
#dataframe['edad'] = (dataframe['edad'] - dataframe['edad'].mean()) / dataframe['edad'].std()
#dataframe['edad'] = dataframe['edad'].round(2)

In [ ]:
#dataframe['años_escolaridad'] = (dataframe['años_escolaridad'] - dataframe['años_escolaridad'].mean()) / dataframe['años_escolaridad'].std()
#dataframe['años_escolaridad'] = dataframe['años_escolaridad'].round(2)

# variable a predecir: 
+ ingreso_total_lab

# Variables independientes
+ sexo (listo)
+ dominio (establece si el individuo pertenece a una villa de emergencia o al resto de la ciudad) cat (klusto)
+ situacion_conyugal cat 
+ edad num
+ afiliacion_salud cat
+ años_escolaridad num
+ hijos_nacidos_vivos cat (listo)



In [ ]:
formula_base = """ingreso_total_lab ~ ingreso_total_no_lab + edad + años_escolaridad + resto_de_la_ciudad + hijos_nacidos_no + 
sexo_varon + casado_a + divorciado_a + separado_a_de_union_o_matrimonio +soltero_a + unido_a + viudo_a +
Solo_obra_social + solo_plan_de_medicina_prepaga_por_contratacion_voluntaria + solo_prepaga_o_mutual_via_OS + solo_sistema_publico"""

modelo_base = smf.ols(formula=formula_base, data=dataframe)

In [ ]:
resultado_base = modelo_base.fit()
resultado_base.summary()

# Ajustar modelo

In [ ]:
formula_ajustada = """ingreso_total_lab ~ + edad + años_escolaridad +  hijos_nacidos_no + sexo_varon + casado_a + divorciado_a + separado_a_de_union_o_matrimonio + soltero_a + unido_a + solo_prepaga_o_mutual_via_OS + solo_sistema_publico"""

In [ ]:
modelo_ajustado = smf.ols(formula=formula_ajustada, data=dataframe)

In [ ]:
resultado_ajustado = modelo_ajustado.fit()
resultado_ajustado.summary()

# Residuos

In [ ]:
dataframe['pred'] = resultado_ajustado.predict(dataframe)
dataframe['residuos'] = dataframe['ingreso_total_lab']-dataframe['pred']


In [ ]:
dataframe['residuos'] = dataframe['residuos'].round(2)

In [ ]:
sns.scatterplot(x=dataframe['ingreso_total_lab'], y =dataframe['residuos'])
plt.axhline(y=0, color='black', linestyle='--')
plt.xlabel('ingreso_total_lab')
plt.ylabel('Residuals')
plt.title('Grafico de residuos')
plt.show()

In [ ]:
dataframe['residuos'].describe()

In [ ]:
plt.figure(figsize=(40, 40))

for idx, col in enumerate(numerical_cols, start=1):  
    plt.subplot(10, 5, idx)
    plt.scatter(data=dataframe, x=col, y='ingresos_totales', alpha=.3)
    plt.title(col)